In [1]:
pip install mysql-connector-python

In [2]:
pip install mysqlclient


Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd
import mysql.connector as db
import numpy as np


In [4]:
from datetime import datetime

In [5]:
mydb = db.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="your_password",
    database="sudesh1")

mycursor = mydb.cursor()

In [30]:
# Loading GitHub Personal Access Token
with open("token.txt", "r") as file:
    GITHUB_TOKEN = file.read().strip() 

# GitHub API URL
GITHUB_API_URL = "https://api.github.com/search/repositories"

# Headers for authentication
authentication = {"Authorization": f"Token {GITHUB_TOKEN}"}

# List of 10 currently trending topics in the data world
topics = ["machine learning", "data visualization", "deep learning",
          "natural language processing", "artificial intelligence", "hypothesis testing",
          "time series analysis", "amazon sagemaker", "tensorflow", "Generative AI"]

# Function to fetch repositories for a specific topic with pagination
def fetch_repositories(topic, max_repos=100):
    repositories = []
    page = 1
    while len(repositories) < max_repos:
        parameter = {
            "q": topic,  # Query parameter for searching repositories by topic
            "sort": "stars",  # Sorting by stars to get the most popular repositories
            "order": "desc",
            "per_page": 100,  # Number of repositories per page
            "page": page  # Pagination control
        }
        response = requests.get(GITHUB_API_URL, headers=authentication, params=parameter)
        if response.status_code == 200:
            repos = response.json()["items"]
            repositories.extend(repos)
            if len(repos) < 100:  # If fewer than 100 repos are returned, exit the loop
                break
        else:
            print(f"Failed to fetch data for {topic}: {response.status_code}")
            break
        page += 1

    return repositories[:max_repos]  # Return only up to the max_repos specified

# List to store the extracted repository data
repositories_data = []

# Extract data for each topic
for topic in topics:
    print(f"Fetching repositories for topic: {topic}")
    repos = fetch_repositories(topic)
    for repo in repos:
        repositories_data.append({
            "Repository Name": repo["name"],
            "Owner": repo["owner"]["login"],
            "Description": repo["description"],
            "URL": repo["html_url"],
            "Programming Language": repo["language"],
            "Creation Date": repo["created_at"],
            "Last Updated Date": repo["updated_at"],
            "Number of Stars": repo["stargazers_count"],
            "Number of Forks": repo["forks_count"],
            "Number of Open Issues": repo["open_issues_count"],
            "License Type": repo["license"]["name"] if repo["license"] else "No License"
        })

# Convert the data to a DataFrame
df = pd.DataFrame(repositories_data)
df.head()

Fetching repositories for topic: machine learning
Fetching repositories for topic: data visualization
Fetching repositories for topic: deep learning
Fetching repositories for topic: natural language processing
Fetching repositories for topic: artificial intelligence
Fetching repositories for topic: hypothesis testing
Fetching repositories for topic: time series analysis
Fetching repositories for topic: amazon sagemaker
Fetching repositories for topic: tensorflow
Fetching repositories for topic: Generative AI


,Repository Name,Owner,Description,URL,Programming Language,Creation Date,Last Updated Date,Number of Stars,Number of Forks,Number of Open Issues,License Type
0,tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,https://github.com/tensorflow/tensorflow,C++,2015-11-07T01:19:20Z,2024-10-07T05:02:16Z,185871,74232,5056,Apache License 2.0
1,transformers,huggingface,🤗 Transformers: State-of-the-art Machine Learn...,https://github.com/huggingface/transformers,Python,2018-10-29T13:56:00Z,2024-10-07T04:46:24Z,133004,26538,1470,Apache License 2.0
2,ML-For-Beginners,microsoft,"12 weeks, 26 lessons, 52 quizzes, classic Mach...",https://github.com/microsoft/ML-For-Beginners,HTML,2021-03-03T01:34:05Z,2024-10-07T05:17:41Z,69335,14346,9,MIT License
3,funNLP,fighting41love,中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...,https://github.com/fighting41love/funNLP,Python,2018-08-21T11:20:39Z,2024-10-07T05:11:23Z,68067,14440,28,No License
4,awesome-machine-learning,josephmisiti,A curated list of awesome Machine Learning fra...,https://github.com/josephmisiti/awesome-machin...,Python,2014-07-15T19:11:19Z,2024-10-07T05:25:49Z,65617,14596,7,Other


In [31]:
# Data Cleaning
df.shape

(1000, 11)

In [32]:
df.info() # show full info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Repository Name        1000 non-null   object
 1   Owner                  1000 non-null   object
 2   Description            984 non-null    object
 3   URL                    1000 non-null   object
 4   Programming Language   867 non-null    object
 5   Creation Date          1000 non-null   object
 6   Last Updated Date      1000 non-null   object
 7   Number of Stars        1000 non-null   int64 
 8   Number of Forks        1000 non-null   int64 
 9   Number of Open Issues  1000 non-null   int64 
 10  License Type           1000 non-null   object
dtypes: int64(3), object(8)
memory usage: 86.1+ KB


In [9]:
df.isna().sum() # check null

Repository Name             0
Owner                       0
Description               650
URL                         0
Programming Language     1042
Creation Date               0
Last Updated Date           0
Number of Stars             0
Number of Forks             0
Number of Open Issues       0
License Type                0
dtype: int64

In [10]:
df['Creation Date'] = pd.to_datetime(df['Creation Date']) # convert data type
df['Last Updated Date'] = pd.to_datetime(df['Last Updated Date'])

# HANDLING NULL VALUES

df = df.replace({np.nan: None})
df['Description'].fillna('No Description', inplace=True)
df['License Type'].fillna('No License', inplace=True)

# Preview cleaned data
print(df.head())

            Repository Name           Owner  \
0                tensorflow      tensorflow   
1              transformers     huggingface   
2          ML-For-Beginners       microsoft   
3                    funNLP  fighting41love   
4  awesome-machine-learning    josephmisiti   

                                         Description  \
0  An Open Source Machine Learning Framework for ...   
1  🤗 Transformers: State-of-the-art Machine Learn...   
2  12 weeks, 26 lessons, 52 quizzes, classic Mach...   
3  中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...   
4  A curated list of awesome Machine Learning fra...   

                                                 URL Programming Language  \
0           https://github.com/tensorflow/tensorflow                  C++   
1        https://github.com/huggingface/transformers               Python   
2      https://github.com/microsoft/ML-For-Beginners                 HTML   
3           https://github.com/fighting41love/funNLP               Pytho

C:\Users\DELL\AppData\Local\Temp\ipykernel_10740\2105133990.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Description'].fillna('No Description', inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_10740\2105133990.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [11]:
df["Description"].sample(3)

9629    A Study in Artificial Intelligence - Simple sc...
4418                        『PythonによるAIプログラミング入門』のリポジトリ 
2100    The easiest way to use deep metric learning in...
Name: Description, dtype: object

In [12]:
max_description_length = df['Description'].str.len().max()
max_description_length

241893

In [13]:
df['Description'] = df['Description'].str[:255]


In [14]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9511 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Repository Name        9511 non-null   object             
 1   Owner                  9511 non-null   object             
 2   Description            9511 non-null   object             
 3   URL                    9511 non-null   object             
 4   Programming Language   8557 non-null   object             
 5   Creation Date          9511 non-null   datetime64[ns, UTC]
 6   Last Updated Date      9511 non-null   datetime64[ns, UTC]
 7   Number of Stars        9511 non-null   int64              
 8   Number of Forks        9511 non-null   int64              
 9   Number of Open Issues  9511 non-null   int64              
 10  License Type           9511 non-null   object             
dtypes: datetime64[ns, UTC](2), int64(3), object(6)
memory usage: 

In [15]:
df["Programming Language"].value_counts()


Programming Language
Jupyter Notebook    2966
Python              2843
JavaScript           473
R                    299
TypeScript           292
                    ... 
VBA                    1
HLSL                   1
Rich Text Format       1
NetLogo                1
V                      1
Name: count, Length: 107, dtype: int64

In [16]:
# Find NaN values in the 'Programming Language' column
total_nan_val = df['Programming Language'].isna().sum()
total_nan_val

954

Since the value counts of Python and Jupyter Notebook are close
Randomly assign the remaining NaN values with Python or Jupyter Notebook based on the ratio

In [17]:
fill_values = ['Python', 'Jupyter Notebook']

nan_val = df['Programming Language'].isna()

# Randomly select between 'Python' and 'Jupyter Notebook' to fill NaN values
df.loc[nan_val, 'Programming Language'] = np.random.choice(fill_values, size=nan_val.sum())

In [18]:
df.shape


(9511, 11)

In [19]:
df.isna().sum()

Repository Name          0
Owner                    0
Description              0
URL                      0
Programming Language     0
Creation Date            0
Last Updated Date        0
Number of Stars          0
Number of Forks          0
Number of Open Issues    0
License Type             0
dtype: int64

In [20]:
unique_number = []
for i in df.columns:
    x = df[i].value_counts().count()
    unique_number.append(x)

pd.DataFrame(unique_number, index = df.columns, columns = ["Total Unique Values"])

,Total Unique Values
Repository Name,8883
Owner,7356
Description,8793
URL,9508
Programming Language,107
Creation Date,9508
Last Updated Date,9484
Number of Stars,2262
Number of Forks,1154
Number of Open Issues,368


In [21]:
df.shape

(9511, 11)

Seems some of the Description column contains other languages so filtering only English for better view.

In [22]:
pip install langdetect


In [23]:
df.to_csv('github_repositor.csv', index=False)
print(df.head)

<bound method NDFrame.head of                                Repository Name                 Owner  \
0                                   tensorflow            tensorflow   
1                                 transformers           huggingface   
2                             ML-For-Beginners             microsoft   
3                                       funNLP        fighting41love   
4                     awesome-machine-learning          josephmisiti   
...                                        ...                   ...   
9995               GENERATIVE_AI_DEV_INDONESIA  GlobalCreativeApkDev   
9996           Generative-AI-Art-For-Beginners       PacktPublishing   
9997                   Generative-AI-on-BigOne                 wuloi   
9998               LangChain-for-Generative-AI       TirendazAcademy   
9999  generativeai-aiml-capital-market-samples           aws-samples   

                                            Description  \
0     An Open Source Machine Learning Framewor

In [28]:
import mysql.connector
import pandas as pd
import numpy as np

# Load CSV 
df = pd.read_csv("github_repositor.csv")

# Convert NaN to None 
df = df.replace({np.nan: None})

df['Creation Date'] = pd.to_datetime(df['Creation Date']).dt.date
df['Last Updated Date'] = pd.to_datetime(df['Last Updated Date']).dt.date



# Create the repositories table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS reposit (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Repository_Name VARCHAR(255) NOT NULL,
    Owner VARCHAR(255) NOT NULL,
    Description TEXT,
    URL VARCHAR(255) NOT NULL,
    Programming_Language VARCHAR(50),
    Creation_Date DATE,
    Last_Updated_Date DATE,
    Number_of_Stars INT,
    Number_of_Forks INT,
    Number_of_Open_Issues INT,
    License_Type VARCHAR(100)
);
"""
mycursor.execute(create_table_query)

# Insert DataFrame into MySQL table
insert_query = """
    INSERT INTO reposit (
        Repository_Name, 
        Owner, 
        Description, 
        URL, 
        Programming_Language, 
        Creation_Date, 
        Last_Updated_Date, 
        Number_of_Stars, 
        Number_of_Forks, 
        Number_of_Open_Issues, 
        License_Type
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df.iterrows():
    val = (
        row['Repository Name'], 
        row['Owner'], 
        row['Description'], 
        row['URL'], 
        row['Programming Language'], 
        row['Creation Date'], 
        row['Last Updated Date'], 
        row['Number of Stars'], 
        row['Number of Forks'], 
        row['Number of Open Issues'], 
        row['License Type']
    )
    
    try:
        mycursor.execute(insert_query, val)
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Commit the transaction
mydb.commit()
print("Data saved to MySQL database!")


Data saved to MySQL database!


In [29]:
# Example Analysis

# Top 5 prog lang by the number of repositories
top_languages = df['Programming Language'].value_counts().head(5)
print("Top 5 Programming Languages:")
print(top_languages)

# Top 5 repositories with the most stars
top_starred_repos = df[['Repository Name', 'Number of Stars']].sort_values(by='Number of Stars', ascending=False).head(5)
print("\nTop 5 Starred Repositories:")
print(top_starred_repos)

# Average number of stars by programming language
avg_stars_per_language = df.groupby('Programming Language')['Number of Stars'].mean().sort_values(ascending=False)
print("\nAverage Stars per Language:")
print(avg_stars_per_language.head(5))

Top 5 Programming Languages:
Programming Language
Jupyter Notebook    3442
Python              3321
JavaScript           473
R                    299
TypeScript           292
Name: count, dtype: int64

Top 5 Starred Repositories:
       Repository Name  Number of Stars
0           tensorflow           185871
1         transformers           133004
1000                d3           108556
7803            models            76997
2     ML-For-Beginners            69335

Average Stars per Language:
Programming Language
Markdown    5709.000000
Jsonnet     4164.000000
MDX         4061.000000
Shell       3154.439024
C++         2777.833333
Name: Number of Stars, dtype: float64
